# PRAD model extension testing

Here we will experiment with adding new features based on DE genes. The DE-genes have been filtered and ranked based on feature elimination utilising RSFs.

In [22]:
library(tidyverse)
library(survival)
library(survminer)
library(glmnet)
library(WriteXLS)
library(ggfortify)
library(circlize)
library(ComplexHeatmap)
library(parallel)
library(broom)
library(survcomp)
library(survivalROC)
library(gtsummary)
source("../getTCGAData.R")
source("../preprocessTCGAData.R")
source("../KM_analysis.R")
source("../Heatmaps.R")
source("../enet.R")
library("readxl")

## Functions 

In [23]:
#
#
#
# Helper function for fixing variable names 
fixVarNames = function(x){
    print(x[1])
    if (str_detect(x, "Gender.clin")) {
        return("Gender")
    } else if (str_detect(x, "Tumor.stage.clin")){
        return("Tumor.stage")
    } else if (str_detect(x,".cn")){
        return(str_extract(x, "\\w+.cn"))
    } else if (str_detect(x, "Gleason.group.clin")){ 
        return("Gleason.group.clin")
    } else {
        return(x)
    }
}


# 
# Function fits a cox regression model
# 
fitCoxModel = function(data, end.point, features){
    
    # Expand to variable name
    end_point_time = paste0(end.point, ".time.clin")
    end_point_event = paste0(end.point, ".clin")

    # Generate a survival formula object 
    survExpression = paste0("Surv(", end_point_time, ", " , end_point_event, ")")
    f <- as.formula(paste(survExpression, paste(features, collapse = " + "), sep = " ~ "))
    
    model.fit = coxph(f, data = data)
    return(model.fit)
}

## Load processed data

In [24]:
tcga.dataset.merged = readRDS("/lustre/projects/landstrom_core/data/rdata_revised/manuscript_work//PRAD//tcga.dataset_merged.rds")

Convert - to _

In [25]:
colnames(tcga.dataset.merged$PFI$train) = str_replace(colnames(tcga.dataset.merged$PFI$train), "-", "_")
colnames(tcga.dataset.merged$PFI$validation) =  str_replace(colnames(tcga.dataset.merged$PFI$validation), "-", "_")

## Load original model and extracting features

In [26]:
# Loading original features 
original.model.features = read_excel("/lustre/projects/landstrom_core/results/prognostic_model_development_revised/models_by_cancer_type/PRAD/Penalized_Cox_risk_prediction/customer_features/With_clinical_features/PFI/Active_covariates_in_lambda.min_model.xlsx")

New names:
• `` -> `...1`


In [27]:
original.model.features

...1,coef,HR
<chr>,<dbl>,<dbl>
Age.clin,2.777648e-06,1.0000028
Gleason.group.clinGleason_group_2,9.138624e-01,2.4939365
KIF23.exp,3.846910e-01,1.4691603
PTEN.exp,-4.741249e-01,0.6224295
VPS4B.exp,-2.241524e-01,0.7991933
AURKA.exp,-4.645377e-01,0.6284256
AURKB.exp,4.394660e-01,1.5518783
TP53.cn-1,7.668698e-02,1.0797041


In [28]:
# Feature names to be selected 
selected.features = map_chr(unlist(original.model.features[,1]), fixVarNames) %>% as.vector()

[1] "Age.clin"
[1] "Gleason.group.clinGleason_group_2"
[1] "KIF23.exp"
[1] "PTEN.exp"
[1] "VPS4B.exp"
[1] "AURKA.exp"
[1] "AURKB.exp"
[1] "TP53.cn-1"


## Refitting the original model using standard cox-regression

In [29]:
# Construct the clinical end points 
end_point_event = paste0("PFI", ".clin")
end_point_time = paste0("PFI", ".time.clin")
    
selected.columns = c(end_point_event, end_point_time, selected.features)

In [30]:
# Input data for cox-model 
input.training = tcga.dataset.merged[["PFI"]]$train %>% dplyr::select(all_of(selected.columns))

# Fit the cox-model 
pcox.ref.fit.pfi.original = fitCoxModel(input.training, "PFI", selected.features)

# Predict the risk scores for validation data 
rel.risk = predict(object = pcox.ref.fit.pfi.original, 
                   newdata = tcga.dataset.merged$PFI$validation[,selected.features], 
                   type = "risk")

#
# Evaluate 
#

# Stratify validation data into two groups based on the fitted relative risk
y.data <- tcga.dataset.merged$PFI$validation[paste0("PFI", c(".clin",".time.clin"))]
colnames(y.data) = c("status","time")

# TEST new function for calculating the C-index
cindex.ref.valid = concordance.index(rel.risk, 
                                        y.data$time, 
                                        y.data$status,
                                        na.rm = TRUE)

# Plot KM and extract the p-value  
KM.valid.ref.by.risk = plotKMbyRelativeRisk(data = y.data, 
                                                     rel.risk = rel.risk)

In [31]:
# Modify the labels 

colnames(tcga.dataset.merged[["PFI"]]$train) = str_replace(colnames(tcga.dataset.merged[["PFI"]]$train), "-", "_")
colnames(tcga.dataset.merged[["PFI"]]$validation) = str_replace(colnames(tcga.dataset.merged[["PFI"]]$validation), "-", "_")

## Load the feature elimination results

In [32]:
de.feat.elim.res = readRDS("/lustre/projects/landstrom_core/data/rdata/manuscript_work/revisions/PRAD/Feature_elimination_res_PFI_DE.rds")

In [33]:
de.feat.elim.res$Features

Variable,Importance
<chr>,<dbl>
CDC25C.exp,0.012259246
AMH.exp,0.011619466
RRM2.exp,0.008777480
KIF20A.exp,0.008335818
ESPL1.exp,0.007593893
DLGAP5.exp,0.007328550
AURKA.exp,0.007244444
FMOD.exp,0.007011241
CDC20.exp,0.006980378


In [34]:
genes.to.add = de.feat.elim.res$Optimal.var$Variable

Remove features already in model

In [36]:
genes.to.add = genes.to.add[!(genes.to.add %in% selected.features)]

In [37]:
genes.to.add = str_replace(genes.to.add, "-", "_")

In [38]:
# Store results 
step.wise.results = matrix(nrow = 0, ncol = 2) %>% as.data.frame() 
colnames(step.wise.results) = c("KM.pvalue","C.index")

In [39]:
step.wise.results = as.data.frame(lapply(step.wise.results, as.numeric))

In [40]:
selected.columns.stepwise = selected.columns

In [41]:
for (i in 1:length(genes.to.add)){
    new.feature = genes.to.add[i]
    
    # Test adding one gene
    selected.columns.stepwise = c(selected.columns.stepwise, new.feature)
    
    # Input data for cox-model 
    input.training = tcga.dataset.merged[["PFI"]]$train[,colnames(tcga.dataset.merged[["PFI"]]$train) %in% selected.columns.stepwise]

    # The features 
    selected.features.stepwise = selected.columns.stepwise[-c(1,2)]
    
    # Fit the cox-model 
    pcox.ref.fit.pfi = fitCoxModel(input.training, "PFI", selected.features.stepwise)

    # Predict the risk scores for validation data 
    rel.risk = predict(object = pcox.ref.fit.pfi , 
                   newdata = tcga.dataset.merged$PFI$validation[,selected.features.stepwise], 
                   type = "risk")

    #
    # Evaluate 
    #

    # Stratify validation data into two groups based on the fitted relative risk
    y.data <- tcga.dataset.merged$PFI$validation[paste0("PFI", c(".clin",".time.clin"))]
    colnames(y.data) = c("status","time")

    # TEST new function for calculating the C-index
    cindex.ref.valid = concordance.index(rel.risk, 
                                        y.data$time, 
                                        y.data$status,
                                        na.rm = TRUE)

    # Plot KM and extract the p-value  
    KM.valid.ref.by.risk = plotKMbyRelativeRisk(data = y.data, 
                                                     rel.risk = rel.risk)
    
    
    res = data.frame(KM.pvalue = KM.valid.ref.by.risk$table$Pvalue.pval,
                     C.index = cindex.ref.valid$c.index,
                     "C-index CI" = paste0("(", round(cindex.ref.valid$lower, 4), " - ",  
                                                round(cindex.ref.valid$upper, 4), ")")
                     )
    
    step.wise.results = rbind(step.wise.results, res)
    
    #print(cindex.ref.valid$c.index)
    #print(KM.valid.ref.by.risk$table)
    
}

Find the minimal p-value and the corresponding C-index

In [42]:
which(step.wise.results$KM.pvalue == min(step.wise.results$KM.pvalue))

[1] 8

In [45]:
step.wise.results[8,]

,KM.pvalue,C.index,C.index.CI
,<dbl>,<dbl>,<chr>
8,0.001966972,0.6249342,(0.3853 - 0.8158)


The feature list : 

In [46]:
genes.to.add[1:8]

[1] "FMOD.exp"            "TICRR.exp"           "AMH.exp"            
[4] "OIP5.exp"            "DMBX1.exp"           "ENSG00000235978.exp"
[7] "CSAG1.exp"           "ENSG00000254548.exp"

In [44]:
step.wise.results

KM.pvalue,C.index,C.index.CI
<dbl>,<dbl>,<chr>
0.022556489,0.6165176,(0.3778 - 0.8098)
0.010569691,0.6217780,(0.3825 - 0.8136)
0.007750568,0.6196739,(0.3806 - 0.8121)
0.002008449,0.6170437,(0.3782 - 0.8102)
0.002116817,0.6223041,(0.3829 - 0.8139)
0.002522324,0.6280905,(0.3881 - 0.8181)
0.002522324,0.6338769,(0.3933 - 0.8222)
0.001966972,0.6249342,(0.3853 - 0.8158)
0.003726015,0.6249342,(0.3853 - 0.8158)
